***


# Projet : Reed-Salomon, QR code et correction d’erreur
***
***


Jean-Loup Mellion  
Gatien Da Rocha

In [118]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

# Rappel interpolation polynomiale

Avant de s'attaquer a Reed-Salomon, il est important de rappeler le principe de l'interpolation polynomiale. L'interpolation polynomiale est une méthode mathématique permettant de trouver un polynôme qui passe par un ensemble de points donnés.

Pour cela, nous allons utiliser la méthode de Lagrange. Cette méthode permet de trouver un polynôme de degré n-1 passant par n points donnés.

### Polynôme de Lagrange

In [119]:
def polynome_base_lagrange(x_array: np.ndarray, y_array: np.ndarray, var_x: float, i: int) -> float:
    """
    Cette fonction calcule le polynome de base de lagrange, c'est à dire le produit des termes (x-x_j)/(x_i-x_j) pour j != i
    :param x_array: np.ndarray, les valeurs de x
    :param y_array: np.ndarray, les valeurs de y
    :param var_x: float, la valeur de x pour laquelle on veut calculer le polynome de base
    :param i: int, l'indice du polynome de base
    :return: float, le polynome de base de lagrange
    """
    res = 1
    for j in range(len(x_array)):
        if j != i:
            res *= (var_x-x_array[j])/(x_array[i]-x_array[j])
    return y_array[i]*res

In [120]:
def poly_lagrange(x_array: np.ndarray, y_array: np.ndarray, var_x: float) -> float:
    """
    Cette fonction calcule le polynome de lagrange pour un x donné, c'est à dire la somme des polynomes de base de lagrange
    :param x_array: np.ndarray, les valeurs de x
    :param y_array: np.ndarray, les valeurs de y
    :param var_x: float, la valeur de x pour laquelle on veut calculer le polynome de lagrange
    """
    sum = 0
    for i in range(len(x_array)):
        sum += polynome_base_lagrange(x_array,y_array,var_x,i)
    return sum

Nous allons maintenant tester la méthode de Lagrange sur un exemple simple.

In [121]:
# On défnit le message à envoyer
message = [6, 9, 4, 2, 0]

# On encode le message
def encode_message(message: list, t: int) -> tuple:
    """
    Cette fonction encode un message en rajoutant de la redondance pour pouvoir corriger des erreurs
    :param message: list, le message à encoder
    :param t: int, le nombre d'erreurs que l'on veut pouvoir corriger
    :return: tuple, les valeurs de x et de y
    """
    x_array = []
    for i in range(len(message)):
        x_array.append(i+1)

    y_array = message
    for i in range(t):
        y_array.append(poly_lagrange(x_array, y_array, len(x_array)+i+1))

    for i in range(t):
        x_array.append(len(x_array)+1)
    return x_array, y_array

x_array, y_array = encode_message(message, 2)

print("x_array:", x_array)
print("y_array:", y_array)

x_array: [1, 2, 3, 4, 5, 6, 7]
y_array: [6, 9, 4, 2, 0, -19.0, -86.0]


In [122]:
# Ensuite on peut retirer 1 ou 2 valeurs du message pour simuler des erreurs
message_erreur = y_array.copy()
message_erreur[0] = 0
message_erreur[2] = 0

print("message_erreur:", message_erreur)

# On doit localiser les erreurs (pour les retirer des valeurs de x, y pour recalculer le polynome de lagrange)
def localiser_erreurs(x_array: np.ndarray, y_array: np.ndarray, message: np.ndarray, t: int) -> np.ndarray:
    """
    Cette fonction localise les erreurs dans le message
    :param x_array: np.ndarray, les valeurs de x
    :param y_array: np.ndarray, les valeurs de y
    :param message: np.ndarray, le message reçu
    :param t: int, le nombre d'erreurs à corriger
    :return: np.ndarray, les indices des erreurs
    """
    erreurs = []
    for i in range(len(message)):
        if message[i] != poly_lagrange(x_array, y_array, i+1):
            erreurs.append(i)
    return erreurs

erreurs = localiser_erreurs(x_array, y_array, message_erreur, t)
print("erreurs localisées:", erreurs)

# On corrige les erreurs en recalculant les valeurs des erreurs
def corriger_erreurs(x_array: np.ndarray, y_array: np.ndarray, message: np.ndarray, erreurs: np.ndarray) -> np.ndarray:
    """
    Cette fonction corrige les erreurs dans le message
    :param x_array: np.ndarray, les valeurs de x
    :param y_array: np.ndarray, les valeurs de y
    :param message: np.ndarray, le message reçu
    :param erreurs: np.ndarray, les indices des erreurs
    :return: np.ndarray, le message corrigé
    """
    for erreur in erreurs:
        message[erreur] = poly_lagrange(x_array, y_array, erreur+1)
    return message

message_corrige = corriger_erreurs(x_array, y_array, message_erreur, erreurs)
print("message_corrigé:", message_corrige)

message_erreur: [0, 9, 0, 2, 0, -19.0, -86.0]
erreurs localisées: [0, 2]
message_corrigé: [6.0, 9, 4.0, 2, 0, -19.0, -86.0]


# Reed-Salomon

# Application dans les QR codes et autres en correction d'erreur